In [1]:
#KÜTÜPHANELER
#Veri İşleme ve Sayısal Hesaplama
import pandas as pd
import numpy as np

# Ön İşleme ve Dönüştürme
from sklearn.preprocessing import StandardScaler, LabelEncoder

#Özellik Seçimi
from sklearn.feature_selection import mutual_info_classif

#Makine Öğrenmesi Modelleri
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

#Model Değerlendirme
from sklearn.model_selection import cross_val_score 

In [2]:
# Veriyi okuma ve ön işleme
def veriyi_yukle_onisle(dosya_yolu):
    # CSV dosyasını oku
    veri = pd.read_csv(dosya_yolu)

    # İlk 5 satırı göster
    print(" İlk 5 Satır:")
    print(veri.head())

    # Veri seti özeti (veri türleri, eksik değer kontrolü)
    print("\n Veri Seti Özeti:")
    print(veri.info())

    # Sayısal sütunlar için istatistiksel özet
    print("\n İstatistiksel Özet:")
    print(veri.describe())

    # Sınıf (etiket) dağılımını göster
    print("\n Sınıf Dağılımı:")
    print(veri['Class'].value_counts())

    # Özellikleri (bağımsız değişkenler) ve hedef sütunu ayır
    X = veri.drop(columns=["Class"])  # 'Class' sütunu çıkarılarak X oluşturulur
    y = veri["Class"]                 # Hedef değişken

    return veri, X, y


In [3]:
# AYKIRI DEĞER SAYIMI (Z-puanı > 3 olanlar)
def aykiri_degerleri_tespit_et(X):
    # Veriyi standartlaştır (ortalama=0, std=1)
    olcekleyici = StandardScaler()
    X_olcekli = olcekleyici.fit_transform(X)
    
    # Z-puanı hesapla (her sütun için): |x - ortalama| / standart sapma
    z_puanlari = np.abs((X_olcekli - X_olcekli.mean(axis=0)) / X_olcekli.std(axis=0))
    
    # Her sütundaki (özellikteki) aykırı değer sayısını hesapla
    aykiri_sayilari = (z_puanlari > 3).sum()
    print("\n Aykırı Değer Sayısı (Her Özellik İçin):")
    print(aykiri_sayilari)
    
    # Her satırda (örnekte) kaç tane aykırı sütun olduğunu hesapla
    satir_aykirilari = (z_puanlari > 3).sum(axis=1)
    print("\n Her Satırdaki Aykırı Özellik Sayısı (İlk 10 Satır):")
    print(satir_aykirilari[:10])
    
    # 2’den fazla aykırı özelliğe sahip kaç örnek olduğunu yazdır
    print(" Birden fazla aykırı özelliğe sahip örnek sayısı:", np.sum(satir_aykirilari > 2))


In [4]:
#Veriyi yükleme
veri, X, y = veriyi_yukle_onisle("Acoustic Features.csv")

#Aykırı değer analizi
aykiri_degerleri_tespit_et(X)

#Hedef (class) değişkeninin sayısal değerlere dönüştürülmesi
etiket_kodlayici = LabelEncoder()
y_kodlanmis = etiket_kodlayici.fit_transform(y)

#MI skorları ile tüm özelliklerin sıralanması
mi_skorlari = mutual_info_classif(X, y_kodlanmis, random_state=42)
mi_sonuclar = pd.DataFrame({
    "Ozellik": X.columns,
    "MI_Skoru": mi_skorlari
}).sort_values(by="MI_Skoru", ascending=False).reset_index(drop=True)


 İlk 5 Satır:
   Class  _RMSenergy_Mean  _Lowenergy_Mean  _Fluctuation_Mean  _Tempo_Mean  \
0  relax            0.052            0.591              9.136      130.043   
1  relax            0.125            0.439              6.680      142.240   
2  relax            0.046            0.639             10.578      188.154   
3  relax            0.135            0.603             10.442       65.991   
4  relax            0.066            0.591              9.769       88.890   

   _MFCC_Mean_1  _MFCC_Mean_2  _MFCC_Mean_3  _MFCC_Mean_4  _MFCC_Mean_5  ...  \
0         3.997         0.363         0.887         0.078         0.221  ...   
1         4.058         0.516         0.785         0.397         0.556  ...   
2         2.775         0.903         0.502         0.329         0.287  ...   
3         2.841         1.552         0.612         0.351         0.011  ...   
4         3.217         0.228         0.814         0.096         0.434  ...   

   _Chromagram_Mean_9  _Chromagram_M

In [5]:
#Modellerin Tanımlanması
modeller = {
    "kNN (En Yakın Komşu)": KNeighborsClassifier(n_neighbors=4),  # 4 komşulu kNN algoritması
    "Karar Ağacı": DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=42),  # Gini ile sınıflandırma yapan karar ağacı
    "Rastgele Orman": RandomForestClassifier(n_estimators=28, max_depth=17, criterion="gini", random_state=42),  # 28 ağaçlı, derinliği 17 olan RF
    "SVM": SVC(kernel='rbf', C=1, gamma='scale', random_state=42),  # RBF çekirdekli SVM
    "Lojistik Regresyon": LogisticRegression(max_iter=1000),  # Maksimum iterasyon sayısı 1000
    "Yalın Bayes": GaussianNB()  # Gauss dağılımına dayalı Naive Bayes sınıflandırıcı
}


In [6]:
#Her model için en uygun özellik sayısının otomatik olarak belirlenmesi
sonuclar = {}

for model_adi, model in modeller.items():
    en_iyi_skor = 0
    en_iyi_ozellik_sayisi = 0

    print(f"\n {model_adi} için özellik sayılarına göre doğruluk oranları:")

    for n in range(5, 51, 5):  # 5’ten 50’ye kadar 5’er artan özellik sayısı dene
        secilen_ozellikler = mi_sonuclar["Ozellik"][:n]
        X_secilen = X[secilen_ozellikler]
        
        # Özelliklerin ölçeklenmesi
        olcekleyici = StandardScaler()
        X_olcekli = olcekleyici.fit_transform(X_secilen)
        
        # 5 katlı çapraz doğrulama ile doğruluk (accuracy) skorlarının alınması
        skorlar = cross_val_score(model, X_olcekli, y_kodlanmis, cv=5, scoring="accuracy")
        ortalama_skor = skorlar.mean()
        
        print(f"  ➤ {n} özellik: Doğruluk (Accuracy) = {ortalama_skor:.4f}")

        #Eğer bu skor en iyiyse güncelle:
        if ortalama_skor > en_iyi_skor:
            en_iyi_skor = ortalama_skor
            en_iyi_ozellik_sayisi = n

    #Sonuçların kaydedilmesi
    sonuclar[model_adi] = (en_iyi_ozellik_sayisi, en_iyi_skor)



 kNN (En Yakın Komşu) için özellik sayılarına göre doğruluk oranları:
  ➤ 5 özellik: Doğruluk (Accuracy) = 0.7125
  ➤ 10 özellik: Doğruluk (Accuracy) = 0.7450
  ➤ 15 özellik: Doğruluk (Accuracy) = 0.7300
  ➤ 20 özellik: Doğruluk (Accuracy) = 0.7225
  ➤ 25 özellik: Doğruluk (Accuracy) = 0.7025
  ➤ 30 özellik: Doğruluk (Accuracy) = 0.7100
  ➤ 35 özellik: Doğruluk (Accuracy) = 0.7000
  ➤ 40 özellik: Doğruluk (Accuracy) = 0.6650
  ➤ 45 özellik: Doğruluk (Accuracy) = 0.7000
  ➤ 50 özellik: Doğruluk (Accuracy) = 0.6975

 Karar Ağacı için özellik sayılarına göre doğruluk oranları:
  ➤ 5 özellik: Doğruluk (Accuracy) = 0.6800
  ➤ 10 özellik: Doğruluk (Accuracy) = 0.6575
  ➤ 15 özellik: Doğruluk (Accuracy) = 0.6825
  ➤ 20 özellik: Doğruluk (Accuracy) = 0.6475
  ➤ 25 özellik: Doğruluk (Accuracy) = 0.7000
  ➤ 30 özellik: Doğruluk (Accuracy) = 0.6950
  ➤ 35 özellik: Doğruluk (Accuracy) = 0.6725
  ➤ 40 özellik: Doğruluk (Accuracy) = 0.6675
  ➤ 45 özellik: Doğruluk (Accuracy) = 0.6650
  ➤ 50 özellik

In [7]:
#MODELLERİN EĞİTİLMESİ
# kNN (En Yakın Komşu)
# En iyi özellik sayısının alınması
knn_ozellik_sayisi = sonuclar["kNN (En Yakın Komşu)"][0] 
X_kNN = X[mi_sonuclar["Ozellik"][:knn_ozellik_sayisi]]

# Özelliklerin ölçekleklenmesi
X_kNN_olcekli = StandardScaler().fit_transform(X_kNN)

# Modelin tanımlanması
knn_modeli = KNeighborsClassifier(n_neighbors=4)

# Doğruluk (accuracy) skorlarının hesaplanmas
knn_dogruluk = cross_val_score(knn_modeli, X_kNN_olcekli, y_kodlanmis, cv=5, scoring="accuracy")

# F1-macro skorlarının hesapla
knn_f1_skorlari = cross_val_score(knn_modeli, X_kNN_olcekli, y_kodlanmis, cv=5, scoring="f1_macro")

# Sonuçlar
print(f"\n kNN ({knn_ozellik_sayisi} özellik)")
print(f"   Doğruluk (Accuracy): {knn_dogruluk.mean():.4f} | Kat Skorları: {knn_dogruluk}")
print(f"   F1-Macro Skoru: {knn_f1_skorlari.mean():.4f} | Kat Skorları: {knn_f1_skorlari}")



 kNN (10 özellik)
   Doğruluk (Accuracy): 0.7450 | Kat Skorları: [0.7875 0.7125 0.725  0.825  0.675 ]
   F1-Macro Skoru: 0.7357 | Kat Skorları: [0.77906586 0.69775874 0.71005027 0.81814465 0.67323389]


In [8]:
#Karar Ağacı
# En iyi özellik sayısının alınması
dt_ozellik_sayisi = sonuclar["Karar Ağacı"][0]  
X_DT = X[mi_sonuclar["Ozellik"][:dt_ozellik_sayisi]]

# Özelliklerin ölçeklenmesi
X_DT_olcekli = StandardScaler().fit_transform(X_DT)

# Mdelini tanımlanması
dt_modeli = DecisionTreeClassifier(criterion='gini', max_depth=10, random_state=42)

# Doğruluk (accuracy) skorlarının hesaplanması
dt_dogruluk = cross_val_score(dt_modeli, X_DT_olcekli, y_kodlanmis, cv=5, scoring="accuracy")

# F1-macro skorlarının hesaplanması
dt_f1_skorlari = cross_val_score(dt_modeli, X_DT_olcekli, y_kodlanmis, cv=5, scoring="f1_macro")

# Sonuçlar
print(f"\n Karar Ağacı ({dt_ozellik_sayisi} özellik)")
print(f"   Doğruluk (Accuracy): {dt_dogruluk.mean():.4f} | Kat Skorları: {dt_dogruluk}")
print(f"   F1-Macro Skoru: {dt_f1_skorlari.mean():.4f} | Kat Skorları: {dt_f1_skorlari}")



 Karar Ağacı (25 özellik)
   Doğruluk (Accuracy): 0.7000 | Kat Skorları: [0.7375 0.65   0.7    0.775  0.6375]
   F1-Macro Skoru: 0.7014 | Kat Skorları: [0.7385751  0.65453872 0.70190772 0.7745975  0.63741401]


In [9]:
# Rastgele Orman 
rf_ozellik_sayisi = sonuclar["Rastgele Orman"][0]
X_RF = X[mi_sonuclar["Ozellik"][:rf_ozellik_sayisi]]
X_RF_olcekli = StandardScaler().fit_transform(X_RF)

rf_model = RandomForestClassifier(n_estimators=28, max_depth=17, criterion="gini", random_state=42)
rf_dogruluk = cross_val_score(rf_model, X_RF_olcekli, y_kodlanmis, cv=5, scoring="accuracy")
rf_f1 = cross_val_score(rf_model, X_RF_olcekli, y_kodlanmis, cv=5, scoring="f1_macro")

print(f"\n Rastgele Orman ({rf_ozellik_sayisi} özellik)")
print(f"   Doğruluk (Accuracy): {rf_dogruluk.mean():.4f} | Skorlar: {rf_dogruluk}")
print(f"   F1-Macro Skoru: {rf_f1.mean():.4f} | Skorlar: {rf_f1}")


 Rastgele Orman (15 özellik)
   Doğruluk (Accuracy): 0.7875 | Skorlar: [0.7875 0.8    0.775  0.8375 0.7375]
   F1-Macro Skoru: 0.7859 | Skorlar: [0.78524579 0.79698303 0.77503168 0.834125   0.73806115]


In [10]:
#SVM 
svm_ozellik_sayisi = sonuclar["SVM"][0]
X_SVM = X[mi_sonuclar["Ozellik"][:svm_ozellik_sayisi]]
X_SVM_olcekli = StandardScaler().fit_transform(X_SVM)

svm_model = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
svm_dogruluk = cross_val_score(svm_model, X_SVM_olcekli, y_kodlanmis, cv=5, scoring="accuracy")
svm_f1 = cross_val_score(svm_model, X_SVM_olcekli, y_kodlanmis, cv=5, scoring="f1_macro")

print(f"\n SVM ({svm_ozellik_sayisi} özellik)")
print(f"   Doğruluk (Accuracy): {svm_dogruluk.mean():.4f} | Skorlar: {svm_dogruluk}")
print(f"   F1-Macro Skoru: {svm_f1.mean():.4f} | Skorlar: {svm_f1}")


 SVM (20 özellik)
   Doğruluk (Accuracy): 0.7950 | Skorlar: [0.85  0.775 0.8   0.825 0.725]
   F1-Macro Skoru: 0.7929 | Skorlar: [0.84817049 0.766708   0.79832709 0.82290902 0.72852526]


In [11]:
# Lojistik Regresyon
lr_ozellik_sayisi = sonuclar["Lojistik Regresyon"][0]
X_LR = X[mi_sonuclar["Ozellik"][:lr_ozellik_sayisi]]
X_LR_olcekli = StandardScaler().fit_transform(X_LR)

lr_model = LogisticRegression(max_iter=1000)
lr_dogruluk = cross_val_score(lr_model, X_LR_olcekli, y_kodlanmis, cv=5, scoring="accuracy")
lr_f1 = cross_val_score(lr_model, X_LR_olcekli, y_kodlanmis, cv=5, scoring="f1_macro")

print(f"\n Lojistik Regresyon ({lr_ozellik_sayisi} özellik)")
print(f"   Doğruluk (Accuracy): {lr_dogruluk.mean():.4f} | Skorlar: {lr_dogruluk}")
print(f"   F1-Macro Skoru: {lr_f1.mean():.4f} | Skorlar: {lr_f1}")


 Lojistik Regresyon (30 özellik)
   Doğruluk (Accuracy): 0.8025 | Skorlar: [0.925  0.7625 0.75   0.825  0.75  ]
   F1-Macro Skoru: 0.8008 | Skorlar: [0.92465604 0.75631313 0.75111073 0.82161285 0.75046904]


In [12]:
#Yalın Bayes
nb_ozellik_sayisi = sonuclar["Yalın Bayes"][0]
X_NB = X[mi_sonuclar["Ozellik"][:nb_ozellik_sayisi]]
X_NB_olcekli = StandardScaler().fit_transform(X_NB)

nb_model = GaussianNB()
nb_dogruluk = cross_val_score(nb_model, X_NB_olcekli, y_kodlanmis, cv=5, scoring="accuracy")
nb_f1 = cross_val_score(nb_model, X_NB_olcekli, y_kodlanmis, cv=5, scoring="f1_macro")

print(f"\n Yalın Bayes ({nb_ozellik_sayisi} özellik)")
print(f"   Doğruluk (Accuracy): {nb_dogruluk.mean():.4f} | Skorlar: {nb_dogruluk}")
print(f"   F1-Macro Skoru: {nb_f1.mean():.4f} | Skorlar: {nb_f1}")


 Yalın Bayes (45 özellik)
   Doğruluk (Accuracy): 0.7625 | Skorlar: [0.875  0.775  0.7    0.7375 0.725 ]
   F1-Macro Skoru: 0.7540 | Skorlar: [0.87417535 0.77121212 0.69444892 0.71064309 0.71976496]
